<a href="https://colab.research.google.com/github/KGuzikowski/AlgTrading/blob/master/Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [1]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3749    0  3749    0     0   8368      0 --:--:-- --:--:-- --:--:--  8349
100  503k  100  503k    0     0   459k      0  0:00:01  0:00:01 --:--:--  459k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3761    0  3761    0     0   9040      0 --:--:-- --:--:-- --:--:--  9040
100  406k  100  406k    0     0   401k      0  0:00:01  0:00:01 --:--:-- 2626k


In [2]:
!pip install python-binance
!pip install binance-client

     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 103 kB 15.7 MB/s 
     |████████████████████████████████| 214 kB 62.3 MB/s 
     |████████████████████████████████| 1.3 MB 41.4 MB/s 
     |████████████████████████████████| 288 kB 60.5 MB/s 
     |████████████████████████████████| 271 kB 62.8 MB/s 
     |████████████████████████████████| 160 kB 57.6 MB/s 


In [3]:
%matplotlib inline

import binance.client
import pandas as pd
import numpy as np
import time
import os
import requests
from pprint import pprint
import subprocess
import datetime as dt
import json
import pandas as pd
import talib

import matplotlib.pyplot as plt
import seaborn as sns

# from xgboost import XGBClassifier
from tqdm import tqdm

# Binance data fetch

In [4]:
class BinanceClient():
    column_names = [
        'open_time',
        'open',
        'high',
        'low',
        'close',
        'vol',
        'close_time',
        'quote_asset_volume',
        'number_of_trades',
        'taker_buy_base_asset_volume',
        'taker_buy_quote_asset_volume',
        'ignore'
    ]

    def __init__(self, api_key, api_secret, interval):
        self.client = binance.client.Client(api_key, api_secret)
        self.INTERVAL = interval

    def polish_kline(self, raw_kline):
        kline_d =  {
            'open_time': raw_kline[0],
            'open': raw_kline[1],
            'close': raw_kline[2],
            'low': raw_kline[3],
            'high': raw_kline[4],
            'vol': raw_kline[5]
        }
        kline_d['open_datetime'] = pd.to_datetime(kline_d['open_time'], unit='ms')
        return kline_d

    def historical_klines_generator(self, instrument, interval, start_dt, end_dt=None):
        for kline in self.client.get_historical_klines_generator(instrument, interval, str(start_dt), str(end_dt) if end_dt else None):
            yield self.polish_kline(kline)

In [5]:
api_key = "vT0XoiF4FbrpHbH04ZEew6A9jOORd79J4RdmId0CTT6wwX97EuY9hPCBcp9F904f"
api_secret = "wcpF4Ne1tNGbq5vPx2oaXrpbE2JnNXhn6tKV6I32uKNdBMuEazRfDPiMJkIVqVr8"

In [6]:
client = BinanceClient(api_key, api_secret, binance.client.Client.KLINE_INTERVAL_5MINUTE)

start_date = dt.datetime(2021,10,7,10,0,0)
instrument = "BTCUSDT"

kline_df = pd.DataFrame(columns=["open","high","low","close","vol"])
for kline in client.historical_klines_generator(instrument, client.INTERVAL, start_date):
    kline_df.loc[kline['open_datetime']] = (kline["open"], kline["high"], kline["low"], kline["close"], kline["vol"])


kline_df.open = kline_df.open.astype(float)
kline_df.close = kline_df.close.astype(float)
kline_df.low = kline_df.low.astype(float)
kline_df.high = kline_df.high.astype(float)
kline_df.vol = kline_df.vol.astype(float)

In [7]:
kline_df

,open,high,low,close,vol
2021-10-07 10:00:00,54300.57,54339.51,54240.98,54432.49,191.72031
2021-10-07 10:05:00,54339.51,54263.05,54200.00,54339.51,175.63492
2021-10-07 10:10:00,54263.04,54142.79,54100.01,54263.05,213.18491
2021-10-07 10:15:00,54143.84,54135.28,54088.39,54199.99,177.47716
2021-10-07 10:20:00,54135.28,54102.98,54046.67,54166.85,183.81866
...,...,...,...,...,...
2021-10-31 16:50:00,60651.45,60657.00,60576.67,60674.28,78.11614
2021-10-31 16:55:00,60657.00,60670.07,60621.06,60700.00,127.41825
2021-10-31 17:00:00,60670.07,60572.98,60542.77,60698.33,114.07823
2021-10-31 17:05:00,60572.98,60518.16,60508.31,60581.80,79.03302


# Indicators

## Indicator base class

In [8]:
class Indicator:
    prev_states = []

    def __init__(self, timeperiod):
        self._timeperiod = timeperiod

    def __function(self):
        raise NotImplementedError()

    def __call__(self, df):
        """
        return:
        1 -> sell
        0 -> do nothing
        -1 -> buy
        """
        return self._function(df)

## Indicators' definitions

### RSI

In [9]:
class RSI(Indicator):
    def __init__(self, timeperiod, lower, upper):
        super().__init__(timeperiod)
        self._lower = lower
        self._upper = upper

    def _function(self, df):
        rsi = talib.RSI(df.close.tail(self._timeperiod + 1), timeperiod=self._timeperiod).iloc[-1]
        
        if rsi >= self._upper:
            return 1
        elif rsi <= self._lower:
            return -1
        else:
            return 0


In [10]:
rsi_20 = RSI(timeperiod=20, lower=35, upper=70)
print(rsi_20(kline_df))
rsi_100 = RSI(timeperiod=100, lower=40, upper=60)
print(rsi_100(kline_df))
rsi_700 = RSI(timeperiod=700, lower=49.5, upper=55)
print(rsi_700(kline_df))

0
0
-1


### Bollinger Bands

In [11]:
class BollingerBands(Indicator):
    def __init__(self, timeperiod, lower=2, upper=3):
        super().__init__(timeperiod)
        self._lower = lower
        self._upper = upper

    def _function(self, df):
        lower, middle, upper = talib.BBANDS(
            df.close.tail(self._timeperiod + 1),
            timeperiod=self._timeperiod,
            nbdevup=self._upper,
            nbdevdn=self._lower,
            matype=0,
        )
        closing =  df.close.iloc[-1]

        if closing >= upper.iloc[-1]:
            return 1
        elif closing <= lower.iloc[-1]:
            return -1
        else:
            return 0

In [12]:
bb = BollingerBands(timeperiod=700, lower=2, upper=3)
bb(kline_df)

1

### EMA small cross EMA big

In [13]:
class EMACross(Indicator):
    """
    If small EMA crosses big EMA from bottom -> buy
    If small EMA crosses big EMA from top -> sell
    """
    def __init__(self, timeperiod_S=100, timeperiod_B=500):
        self._timeperiod_S = timeperiod_S
        self._timeperiod_B = timeperiod_B

    def _function(self, df):
        ma_S = talib.EMA(df.close.tail(self._timeperiod_S + 1), timeperiod=self._timeperiod_S)
        ma_B = talib.EMA(df.close.tail(self._timeperiod_B + 1), timeperiod=self._timeperiod_B)

        ma_S0 = ma_S.iloc[-2]
        ma_S1 = ma_S.iloc[-1]
        ma_B0 = ma_B.iloc[-2]
        ma_B1 = ma_B.iloc[-1]

        if ma_S0 >= ma_B0 and ma_S1 <= ma_B1:
            # cross from top
            return 1
        elif ma_S0 <= ma_B0 and ma_S1 >= ma_B1:
            # cross from bottom
            return -1
        else:
            return 0

In [14]:
ema_cross = EMACross(timeperiod_S=100, timeperiod_B=500)
ema_cross(kline_df)

0

### EMA
Check where close price crosses EMA

In [15]:
class EMA(Indicator):
    """
    If price crosses EMA from bottom -> buy
    If price crosses EMA from top -> sell
    """
    def __init__(self, timeperiod):
        super().__init__(timeperiod)

    def _function(self, df):
        ma = talib.EMA(df.close.tail(self._timeperiod + 1), timeperiod=self._timeperiod)

        ma0 = ma.iloc[-2]
        ma1 = ma.iloc[-1]
        close0 = df.close.iloc[-2]
        close1 = df.close.iloc[-1]

        if ma0 >= close0 and ma1 <= close1:
            # cross from top
            return 1
        elif ma0 <= close0 and ma1 >= close1:
            # cross from bottom
            return -1
        else:
            return 0

In [16]:
ema_100 = EMA(timeperiod=100)
print(ema_100(kline_df))
ema_500 = EMA(timeperiod=500)
print(ema_500(kline_df))

0
0


### Normalized EMA small cross normalized EMA big

In [17]:
class NormalizedEMACross(Indicator):
    """
    If small EMA crosses big EMA from bottom -> buy
    If small EMA crosses big EMA from top -> sell
    """
    def __init__(self, timeperiod_S=100, timeperiod_B=500):
        self._timeperiod_S = timeperiod_S
        self._timeperiod_B = timeperiod_B

    def _function(self, df):
        ma_S = talib.EMA(df.close.tail(self._timeperiod_S + 1), timeperiod=self._timeperiod_S)
        ma_S = (ma_S-ma_S.mean())/ma_S.std()
        ma_B = talib.EMA(df.close.tail(self._timeperiod_B + 1), timeperiod=self._timeperiod_B)
        ma_B = (ma_B-ma_B.mean())/ma_B.std()

        ma_S0 = ma_S.iloc[-2]
        ma_S1 = ma_S.iloc[-1]
        ma_B0 = ma_B.iloc[-2]
        ma_B1 = ma_B.iloc[-1]

        if ma_S0 >= ma_B0 and ma_S1 <= ma_B1:
            # cross from top
            return 1
        elif ma_S0 <= ma_B0 and ma_S1 >= ma_B1:
            # cross from bottom
            return -1
        else:
            return 0

In [18]:
ema_cross = NormalizedEMACross(timeperiod_S=100, timeperiod_B=500)
ema_cross(kline_df)

0

### Normalized EMA smaller/bigger than 0

In [19]:
class NormalizedEMA(Indicator):
    """
    If normalized EMA > 0 -> sell
    If normalized EMA < 0 -> buy
    """
    def __init__(self, timeperiod=200):
        self._timeperiod = timeperiod

    def _function(self, df):
        ma = talib.EMA(df.close.tail(self._timeperiod + 1), timeperiod=self._timeperiod)
        ma = (ma-ma.mean())/ma.std()

        ma = ma.iloc[-1]
        print(ma)

        if ma > 0:
            return 1
        elif ma < 0:
            return -1
        else:
            return 0

In [20]:
ema_cross = NormalizedEMA(timeperiod=200)
ema_cross(kline_df)

-0.707106781187448


-1

### MACD

In [44]:
class MACD(Indicator):
    """
    If small EMA crosses big EMA from bottom -> buy
    If small EMA crosses big EMA from top -> sell
    """
    def __init__(self, fastperiod=200, slowperiod=500, signalperiod=100):
        self._fastperiod = fastperiod
        self._slowperiod = slowperiod
        self._signalperiod = signalperiod

    def _function(self, df):
        macd, macdsignal, _ = talib.MACD(df.close, fastperiod=self._fastperiod, slowperiod=self._slowperiod, signalperiod=self._signalperiod)
        
        macd_S1 = macd.iloc[-2]
        macd_S2 = macd.iloc[-1]
        macdsignal_S1 = macdsignal.iloc[-2]
        macdsignal_S2 = macdsignal.iloc[-1]

        if macd_S1 >= macdsignal_S1 and macd_S2 <= macdsignal_S2:
            # cross from top
            return 1
        elif macd_S1 <= macdsignal_S1 and macd_S2 >= macdsignal_S2:
            # cross from bottom
            return -1
        else:
            return 0

In [45]:
macd = MACD(fastperiod=200, slowperiod=500, signalperiod=100)
macd(kline_df)

0